In [1]:
from google.cloud import bigquery
bq = bigquery.Client()

In [2]:
q = bq.query(
"""
select count(*) as num_rows 
from bigquery-public-data.geo_us_boundaries.counties
""")
q.to_dataframe()

,num_rows
0,3233


In [3]:
no_cache = bigquery.QueryJobConfig(use_query_cache=False)

In [4]:
#q1
q1 = bq.query("""
SELECT geo_id
FROM bigquery-public-data.geo_us_boundaries.counties
WHERE county_name = 'Dane'
""", job_config=no_cache)

q1.to_dataframe()["geo_id"][0]

'55025'

In [5]:
#q2
q2 = bq.query("""
SELECT state_fips_code, COUNT(*) AS num_counties
FROM bigquery-public-data.geo_us_boundaries.counties
GROUP BY state_fips_code
ORDER BY num_counties DESC
LIMIT 5
""", job_config=no_cache)

q2.to_dataframe().set_index('state_fips_code')['num_counties'].to_dict()

{'48': 254, '13': 159, '51': 133, '21': 120, '29': 115}

In [6]:
#q3
iowa_cost = 6.25

cost_q1 = q1.total_bytes_billed / (1024**4) * iowa_cost
cost_q2 = q2.total_bytes_billed / (1024**4) * iowa_cost

cost_dict = {'q1': cost_q1, 'q2': cost_q2}
cost_dict

{'q1': 5.9604644775390625e-05, 'q2': 5.9604644775390625e-05}

In [7]:
dataset = bigquery.Dataset("local-storm-398318.p8")
dataset.location = "US"
dataset = bq.create_dataset(dataset, exists_ok=True)

config = bigquery.LoadJobConfig(source_format="PARQUET", write_disposition="WRITE_TRUNCATE")
url = "gs://cs544-p8/hdma-wi-2021.parquet"
job = bq.load_table_from_uri(url, "local-storm-398318.p8.hdma", job_config=config)
job.result()

LoadJob<project=local-storm-398318, location=US, id=bd6e6f6c-2dbf-4b16-9cd8-c80b19564dd7>

In [8]:
#q4
[ds.dataset_id for ds in bq.list_datasets("local-storm-398318")]

['p8']

In [9]:
#q5
q5 = bq.query("""
    SELECT c.county_name, COUNT(*) AS num_applications
    FROM `local-storm-398318.p8.hdma` AS h
    JOIN bigquery-public-data.geo_us_boundaries.counties AS c
    ON h.county_code = c.geo_id
    GROUP BY c.county_name
    ORDER BY num_applications DESC
    LIMIT 10
""", job_config=no_cache)

q5.to_dataframe().set_index('county_name')['num_applications'].to_dict()

{'Milwaukee': 46570,
 'Dane': 38557,
 'Waukesha': 34159,
 'Brown': 15615,
 'Racine': 13007,
 'Outagamie': 11523,
 'Kenosha': 10744,
 'Washington': 10726,
 'Rock': 9834,
 'Winnebago': 9310}

In [10]:
url = "https://docs.google.com/spreadsheets/d/11UeIBqQylAyNUBsIO54p6WiYJWHayQMfHDbUWq1jGco/"

external_config = bigquery.ExternalConfig("GOOGLE_SHEETS")
external_config.source_uris = [url]
external_config.options.skip_leading_rows = 1
external_config.autodetect = True

table = bigquery.Table("local-storm-398318.p8.applications")
table.external_data_configuration = external_config

table = bq.create_table(table, exists_ok=True)

In [11]:
#q6
q6 = bq.query("""
    SELECT COUNT(*) AS num_applications
    FROM p8.applications
    WHERE income = 40000
""", job_config=no_cache)

q6.to_dataframe()["num_applications"][0]

3

In [12]:
bq.query("""
CREATE OR REPLACE TABLE `local-storm-398318.p8.wi`
AS
SELECT county_name, county_geom
FROM bigquery-public-data.geo_us_boundaries.counties
WHERE state_fips_code = '55'
""", job_config=no_cache).to_geodataframe()

bq.query("""
CREATE OR REPLACE TABLE p8.houses
AS
SELECT *, ST_GEOGPOINT(longitude, latitude) as loc
FROM p8.applications
""", job_config=no_cache).to_geodataframe()

""


In [13]:
#q7
q7 = bq.query("""
    SELECT c.county_name, COUNT(*) AS num_applications
    FROM p8.houses AS h
    JOIN p8.wi AS c
    ON ST_CONTAINS(c.county_geom, h.loc)
    GROUP BY c.county_name       
""", job_config=no_cache)

q7.to_dataframe().set_index('county_name')['num_applications'].to_dict()

{'Outagamie': 1,
 'Door': 4,
 'Sauk': 1,
 'Green Lake': 1,
 'Monroe': 1,
 'Barron': 2,
 'Oneida': 1,
 'Sheboygan': 2,
 'Brown': 2,
 'Columbia': 1,
 'Kewaunee': 1,
 'Walworth': 2,
 'Dane': 9,
 'Bayfield': 1,
 'Marinette': 2,
 'Jefferson': 1}

In [14]:
bq.query("""
    CREATE OR REPLACE MODEL `local-storm-398318.p8.p8_model`
    OPTIONS(MODEL_TYPE="LINEAR_REG",
            INPUT_LABEL_COLS=["loan_amount"])
    AS
    SELECT income, loan_term, loan_amount
    FROM p8.hdma
""")

QueryJob<project=local-storm-398318, location=US, id=66e74f84-90d7-4c29-9a89-53e8774dc713>

In [15]:
import time
while True:
    model_list = [model.model_id for model in bq.list_models("local-storm-398318.p8")]
    if "p8_model" in model_list:  # Hint: use bq.list_models()
        break
    time.sleep(5)

In [16]:
#q8
bq.query("""
    SELECT *
    FROM ML.EVALUATE(MODEL `local-storm-398318.p8.p8_model`, (
        SELECT income, loan_term, loan_amount
        FROM p8.hdma 
    ))
""").to_dataframe()["r2_score"][0]

0.29165412288019577

In [17]:
#q9
bq.query("""
    SELECT *
    FROM ML.WEIGHTS(MODEL `local-storm-398318.p8.p8_model`)
""").to_dataframe().set_index("processed_input").loc["income", "weight"]

0.8057773037176712

In [18]:
#q10
q10 = bq.query("""
    SELECT predicted_loan_amount, actual_loan_amount
    from ML.PREDICT(MODEL `local-storm-398318.p8.p8_model`, 
         (SELECT income, 
         360 AS loan_term, 
         loan_amount AS actual_loan_amount
         FROM p8.applications )
         )
""").to_dataframe()

num_greater = len(q10[q10['actual_loan_amount'] > q10['predicted_loan_amount']])
total = len(q10)
num_greater / total

0.6721311475409836